In [2]:
from bs4 import BeautifulSoup
import urllib

import pandas as pd

import re

from pycountry import countries
import glob

from pysubparser import parser
from pysubparser.cleaners import ascii, formatting

# Data from Wikipedia

In [2]:
# Download the html from Wikipedia for the first 17 seasons
htmls = []
for i in range(1,18):
    resp = urllib.request.urlopen(f"https://en.wikipedia.org/wiki/NCIS_(season_{i})")
    htmls.append(resp.read().decode())

In [6]:
# prepare data structure
head = ["no", "season", "no_in_season", "title", "air_date", "us_viewers", "summary"]
data = {h:[] for h in head}

# parse all the Wiki pages
for i, html in enumerate(htmls):
    soup = BeautifulSoup(html)
    season = i+1
    
    # get the table where the data is
    table = soup.find("table", class_="wikiepisodetable")
    trs = table.tbody.findAll("tr")
    
    # parse the data from the table
    # iterator is used to look at two rows at the same time
    it = iter([["".join(td.stripped_strings) for td in tr] for tr in trs[1:]])
    for tr in it:
        # check for the known different lengths
        data["season"].append(season)
        if len(tr) == 7 and season==1:
            data["no"].append(int(tr[0]))  # use "no. in season" twice
        elif len(tr) == 8 or season==17:
            data["no"].append(int(tr.pop(0)))  # use "no." only once and remove         
        else:
            # if the data is differently structured for any unexpected season
            raise Exception(f"Not expected: len(tr) == {len(tr)} in season {season}")
        
        # going through the cells, parsing the data in the expected data types
        data["no_in_season"].append(int(tr[0]))
        data["title"].append(tr[1].strip('"'))
        try:
            date = re.match(r".*\((.*)\)", tr[4]).groups()[0]
        except AttributeError:
            print(f"weird date in season {season}, no. {tr[0]}: {tr[4]}")
            date = tr[4]
        data["air_date"].append(date)
        viewers = float(re.match(r"(.*)\[.*\]", tr[-1]).groups()[0])
        data["us_viewers"].append(viewers)
        data["summary"].append(next(it)[0])  # get "text" from next row

In [7]:
# create pandas DataFrame from data
df = pd.DataFrame(data, columns=head)
df["air_date"] = [pd.Timestamp(d) for d in df["air_date"]]
df.head()

,no,season,no_in_season,title,air_date,us_viewers,text
0,1,1,1,"""Yankee White""",2003-09-23,13.04,"While onAir Force One, a Navy commander, Ray T..."
1,2,1,2,"""Hung Out to Dry""",2003-09-30,12.08,A Marine (Brian Patrick Wade) dies during a ni...
2,3,1,3,"""Seadog""",2003-10-07,11.26,"When a driver-less boat and several bodies, in..."
3,4,1,4,"""The Immortals""",2003-10-14,11.70,The discovery of a drowned sailor in dress whi...
4,5,1,5,"""The Curse""",2003-10-28,13.50,Gibbs and the team are called in when a mummif...


In [8]:
df.dtypes

no                       int64
season                   int64
no_in_season             int64
title                   object
air_date        datetime64[ns]
us_viewers             float64
text                    object
dtype: object

In [9]:
df.to_pickle("data.pkl")

# Add text from subtitles

In [35]:
# load earlier created data if neaded
if "df" not in globals(): 
    df = pd.read_pickle("data.pkl")

In [36]:
sub_files = glob.glob("../ncis_subs/*/*")
# sort the files based on different file names 
sorted_sub_files = []
for sf in sub_files:
    match = re.match(r".*?([0-9]+)[xEe]([0-9]{2}).*", sf)
    try:
        groups = match.groups()
        s = int(groups[0])
        e = int(groups[1])
    except AttributeError:
        raise AttributeError(f"!!! Didnt work on {sf}")
    sorted_sub_files.append((s, e, sf))
sorted_sub_files.sort(key=lambda x: (x[0], x[1]))

In [32]:
# only take the text and make a list
subtitles = []
for s, e, sf in sorted_sub_files:
    try:
        subs = list(parser.parse(sf))
    except UnicodeDecodeError:
        subs = list(parser.parse(sf, encoding='cp1252'))
    
    subs = ascii.clean(formatting.clean(subs))
    sub_texts = [sub.text for sub in subs]
    
    text = "\n".join(sub_texts)
    subtitles.append(text)

In [37]:
# fill subtitles-list with "" for missing 17th season
diff = len(df) - len(subtitles)
for i in range(diff):
    subtitles.append("")

In [40]:
df["subtitles"] = subtitles

In [43]:
df.to_pickle("../data_subs.pkl")